In [13]:
import constant
import requests
from bs4 import BeautifulSoup as bs
import lxml
import json
import constant
import ast
import re

In [14]:
def get_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
    r = requests.get(url, headers = headers)
    return r

In [15]:
 r = get_html('https://kudago.com/public-api/v1.4/events/?page_size=100&order_by=-publication_date&location=spb&expand=price,place,images,categories,dates,site_url&fields=id,title,price,place,images,dates,categories,site_url')

In [31]:
json_html = r.json()
# список словарей всех мероприятий 
ads = json_html['results']
# сохраняем id первого мероприятия в полученном списке
for ad in ads:
    # производим получение данных из json для БД
    id_parse  = ad['id']
    type_event_kudago = ad['categories'][0]
    type_event = constant.dictonary_event_kudago.get(type_event_kudago, 'Разное')
    img = ad['images'][0]['thumbnails']['640x384']
    title = ad['title']
    cost = ad.get('price', None)
    discounted = '0'
    address = ad.get('place', None)
    if address:
        address_place = address.get('address', None)
        name_place = address.get('title', None)
        address = connect_full_address(name_place, address_place)
    try:
        metro_list = ad['place']['subway'] 
        metro = metro_list.split(', ')[0]
    except:
        metro = None
    link = ad['site_url'] 
    try:
        date_start_info = ad['dates'][0]['start_date']
        date_start = format.kudago_and_kassir_date_format(date_start_info)
    except:
        date_start = None
    try:
        count_date_length = len(ad['dates'])
        if count_date_length == 1:
            date_stop_info =  ad['dates'][0]['end_date'] 
        else:
            date_stop_info =  ad['dates'][count_date_length-1]['start_date']
            date_stop = format.kudago_and_kassir_date_format(date_stop_info)
    except:
        date_stop = None
    if date_start == date_stop:
        date_stop = None
      
    print(id_parse, type_event, img, title, date_start, date_stop, cost, discounted, address, metro, link, '\n', sep=', ')

56c21d3b42d9e5f.jpg, концерт «Бессмертный полк», None, None, от 1300 до 3800 рублей, 0, БКЗ «Октябрьский», Лиговский просп., д. 6, Площадь Восстания, https://kudago.com/spb/event/kontsert-bessmertnyij-polk-maj-2021/, 

191729, Концерт, https://kudago.com/media/thumbs/640x384/images/event/11/76/11768192bc530e03e47ed46b57eeef5b.jpg, концерт Мэвла, None, None, от 900 до 2500 рублей, 0, концертный зал Aurora, Пироговская наб., д. 5/2, Площадь Ленина, https://kudago.com/spb/event/kontsert-mevl-maj-2021/, 

191784, Обучение, https://kudago.com/media/thumbs/640x384/images/event/8d/d3/8dd33dc41ff8344f94a62b01ae9db252.jpg, конференция Data Science Conference от Школы программирования Elbrus Coding Bootcamp, None, None, , 0, None, None, https://kudago.com/online/event/obuchenie-data-science-conference/, 

191780, Развлечения, https://kudago.com/media/thumbs/640x384/images/event/a8/fe/a8fe5d2ca95f213d2a7b70fdb21bc18c.jpg, фестиваль «Корюшка идёт», None, None, , 0, Новая Ладога, ул. Володарского, 

In [73]:
full_address = 'м. Василеостровская, Кожевенная линия, 40, 2-й этаж, пространство «Севкабель Порт»'
if full_address.startswith('м.'):
    sep_full_address = full_address.split(',')
    address = ''.join(sep_full_address[1:]).strip()
    metro = sep_full_address[0]
    print(address)
    print(metro)

Кожевенная линия 40 2-й этаж пространство «Севкабель Порт»
м. Василеостровская


In [28]:
def connect_full_address(name_place, address_place):
    if name_place and address_place:
        name_place += ', '
    full_address = name_place + address_place
    
    return full_address if full_address else None

In [11]:
z = 'балаган-шоу «Жили-были» при участии группы EtnoZapil в «Ленинград Центре»'
x = z.replace(u'\u00ab', '\"').replace(u'\u00bb', '\"')
print(x)

балаган-шоу "Жили-были` при участии группы EtnoZapil в "Ленинград Центре`
